# 02 - Cell Segmentation Pipeline

This notebook walks through the segmentation pipeline used to identify
individual cells (or regions) in XRF elemental maps:

1. Preprocessing and Otsu thresholding
2. Morphological operations (opening, closing, dilation)
3. Connected-component labeling
4. Filtering by area, aspect ratio, and solidity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, morphology, measure, segmentation
from skimage.color import label2rgb

%matplotlib inline

## Synthetic test image

We create a synthetic elemental map with blob-like structures to
demonstrate the pipeline without requiring a real dataset.

In [ ]:
rng = np.random.default_rng(42)
image = np.zeros((256, 256), dtype=np.float64)

# Place Gaussian blobs to simulate cell-like regions
yy, xx = np.mgrid[0:256, 0:256]
for _ in range(15):
    cy, cx = rng.integers(30, 226, size=2)
    sigma = rng.uniform(8, 18)
    intensity = rng.uniform(0.5, 1.0)
    image += intensity * np.exp(-((yy - cy)**2 + (xx - cx)**2) / (2 * sigma**2))

image += 0.05 * rng.standard_normal(image.shape)
image = np.clip(image, 0, None)

plt.imshow(image, cmap="inferno")
plt.title("Synthetic XRF elemental map")
plt.colorbar()
plt.show()

## 1. Otsu thresholding

In [ ]:
threshold = filters.threshold_otsu(image)
binary = image > threshold

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(image, cmap="inferno")
axes[0].set_title("Original")
axes[1].imshow(binary, cmap="gray")
axes[1].set_title(f"Otsu threshold = {threshold:.4f}")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

## 2. Morphological operations

We apply opening to remove small noise, then closing to fill gaps.

In [ ]:
selem = morphology.disk(3)
opened = morphology.binary_opening(binary, selem)
closed = morphology.binary_closing(opened, morphology.disk(5))

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
for ax, img, title in zip(axes, [binary, opened, closed],
                          ["Binary", "After opening", "After closing"]):
    ax.imshow(img, cmap="gray")
    ax.set_title(title)
    ax.axis("off")
plt.tight_layout()
plt.show()

## 3. Connected-component labeling

In [ ]:
labels = measure.label(closed)
print(f"Number of connected components: {labels.max()}")

overlay = label2rgb(labels, image=image, bg_label=0, alpha=0.4)
plt.figure(figsize=(6, 6))
plt.imshow(overlay)
plt.title("Connected components")
plt.axis("off")
plt.show()

## 4. Filtering regions

Remove regions that are too small, too elongated, or non-convex.

In [ ]:
MIN_AREA = 100
MAX_ASPECT = 3.0
MIN_SOLIDITY = 0.6

props = measure.regionprops(labels, intensity_image=image)
filtered_mask = np.zeros_like(labels, dtype=bool)

kept = []
for region in props:
    if region.area < MIN_AREA:
        continue
    aspect = region.major_axis_length / (region.minor_axis_length + 1e-9)
    if aspect > MAX_ASPECT:
        continue
    if region.solidity < MIN_SOLIDITY:
        continue
    filtered_mask[labels == region.label] = True
    kept.append(region)

print(f"Kept {len(kept)} of {len(props)} regions")

filtered_labels = measure.label(filtered_mask)
overlay_filtered = label2rgb(filtered_labels, image=image, bg_label=0, alpha=0.4)
plt.figure(figsize=(6, 6))
plt.imshow(overlay_filtered)
plt.title("Filtered regions")
plt.axis("off")
plt.show()